In [0]:
# !pip install torchsummary
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary
import numpy as np

In [0]:
class LeNet5_with_BN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1)
        self.bn_1 = nn.BatchNorm2d(num_features=6)
        self.max_pool_1  = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        self.bn_2 = nn.BatchNorm2d(num_features=16)
        self.max_pool_2 = nn.MaxPool2d(kernel_size=2)

        self.fc1 = nn.Linear(in_features=16*4*4, out_features=120)
        self.bn_3 = nn.BatchNorm1d(num_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)
        
    def forward(self, xb):
        xb = self.conv1(xb)
        xb = self.bn_1(xb)
        xb = F.relu(xb)
        xb = self.max_pool_1(xb)
        xb = self.conv2(xb)
        xb = self.bn_2(xb)
        xb = F.relu(xb)
        xb = self.max_pool_2(xb)
        xb = xb.view(-1, 16*4*4)
        xb = self.fc1(xb)
        xb = self.bn_3(xb)
        xb = F.relu(xb)
        xb = F.relu(self.fc2(xb))
        xb = self.fc3(xb)
        output = F.log_softmax(xb, dim=1)
        return output

In [0]:
args={}
kwargs={}
args['batch_size']=32
args['test_batch_size']=1000
args['epochs']=10  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.01 #Learning rate is how fast it will decend. 
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).
args['gamma'] = 0.7  #Learning rate step gamma (default: 0.7)

args['seed']=1 #random seed
args['log_interval']=100
args['cuda']=True

device = torch.device("cuda" if args['cuda'] else "cpu")

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['batch_size'], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    epoch_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        epoch_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return np.mean(epoch_loss)

In [6]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [12]:
model = LeNet5_with_BN().to(device)
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
       BatchNorm2d-2            [-1, 6, 24, 24]              12
         MaxPool2d-3            [-1, 6, 12, 12]               0
            Conv2d-4             [-1, 16, 8, 8]           2,416
       BatchNorm2d-5             [-1, 16, 8, 8]              32
         MaxPool2d-6             [-1, 16, 4, 4]               0
            Linear-7                  [-1, 120]          30,840
       BatchNorm1d-8                  [-1, 120]             240
            Linear-9                   [-1, 84]          10,164
           Linear-10                   [-1, 10]             850
Total params: 44,710
Trainable params: 44,710
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.08
Params size (MB): 0.17
Estimated Tot

In [13]:
optimizer = optim.Adadelta(model.parameters(), lr=args['lr'])

# optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])

scheduler = StepLR(optimizer, step_size=1, gamma=args['gamma'])
train_loss = []
test_accuracy = []
for epoch in range(1, args['epochs'] + 1):
    loss = train(args, model, device, train_loader, optimizer, epoch)
    accuracy= test(args, model, device, test_loader)
    scheduler.step()
    train_loss.append(loss)
    test_accuracy.append(accuracy)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.386609
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.163450
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.162674
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.896175
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.629009
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.599639
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.352715
Train Epoch: 1 [22400/60000 (37%)]	Loss: 1.208662
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.127555
Train Epoch: 1 [28800/60000 (48%)]	Loss: 1.003817
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.788215
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.733241
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.666083
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.662612
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.566113
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.661019
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.677127
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.585152
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.576871

Test set: Average loss: 0.3889, Accuracy: 9242/10000 (92.4